In [1]:
from fase2_funciones import *
from Apriori import *
import os
import subprocess

df_bd_preprocesada = pd.read_csv('BD/OnlineRetail_Preprocesada.csv', sep=',', encoding = 'unicode_escape')
df_pivot = df_bd_preprocesada['InvoiceDate'].str.split(expand = True)
df_pivot.rename(columns = {0: 'Fecha', 1: 'Hora'}, inplace=True)
df_bd_preprocesada = df_bd_preprocesada.drop(columns =['Unnamed: 0','InvoiceDate'])
df_bd_preprocesada = pd.concat([df_bd_preprocesada,df_pivot], axis = 1)
df_bd_preprocesada['Monto'] = df_bd_preprocesada['UnitPrice'] * df_bd_preprocesada['Quantity']

In [2]:
#--------------------Llamado a funciones ------------------
articulos_libras_promedio (df_bd_preprocesada)
grupo_cliente(df_bd_preprocesada)
numero_transacciones(df_bd_preprocesada)


-----------------------Articulos/Libras promedio-------------------------------
 
El promedio de articulos por compra es  159.54274541531822 

------------------------------------------------------
 
El promedio de libras gastadas por compra es  293.6456461704423 

---------------------------Clasificación de clientes ---------------------------
 
Cantidad total de clientes 4339
Cantidad de clientes premium 872
Cantidad de clientes normales 3467

---------------------Horas de mayor/menor transacciones ---------------------------------
 
la hora donde más se compra es 12 horas
la hora donde menos se compra es 6 horas


In [3]:
#--------------------Se ejecuta el archivo Apriori.py-------------
a_priori(df_bd_preprocesada)


Aplicando algoritmo A-priori... 


---------------top 20 reglas  -----------------
frozenset({' ', 'FANCY FONT BIRTHDAY CARD'})
frozenset({' BACK DOOR ', 'KEY FOB '})
frozenset({'KEY FOB ', ' SHED'})
frozenset({'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE CHOCOLATE'})
frozenset({'ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE CHOCOLATE'})
frozenset({'ALARM CLOCK BAKELIKE ORANGE', 'ALARM CLOCK BAKELIKE GREEN'})
frozenset({'ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE RED '})
frozenset({'ALARM CLOCK BAKELIKE RED ', 'ALARM CLOCK BAKELIKE IVORY'})
frozenset({'ALARM CLOCK BAKELIKE ORANGE', 'ALARM CLOCK BAKELIKE RED '})
frozenset({'ALARM CLOCK BAKELIKE PINK', 'ALARM CLOCK BAKELIKE RED '})
frozenset({'PAINTED METAL PEARS ASSORTED', 'ASSORTED COLOUR BIRD ORNAMENT'})
frozenset({'BAKING SET SPACEBOY DESIGN', 'BAKING SET 9 PIECE RETROSPOT '})
frozenset({'TOILET METAL SIGN', 'BATHROOM METAL SIGN'})
frozenset({'PINK HAPPY BIRTHDAY BUNTING', 'BLUE HAPPY BIRTHDAY BUNTING'})
frozenset